# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from key import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities=pd.read_csv('../WeatherPy/output_data/cities.csv')
cities.head()


,City,Longitude,Latitude,Temperature,Humidity,Wind,Cloudiness,Country,Date
0,Turochak,87.1224,52.2576,48.38,84,2.51,54,RU,1621119958
1,Airai,125.4092,-8.9266,60.51,80,3.96,79,TL,1621119958
2,Busselton,115.3333,-33.6500,61.00,72,7.00,98,AU,1621119958
3,Agua Prieta,-109.5667,31.3000,84.00,12,10.00,1,MX,1621119959
4,Hong Kong,114.1577,22.2855,86.00,82,8.10,36,HK,1621119875



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities[['Latitude', 'Longitude']].astype(float)

humidity = cities['Humidity'].astype(float)

center = (35,0)

fig = gmaps.figure(center=center, zoom_level=1)

heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=80, 
    point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
top_cities = cities.loc[(cities['Temperature'] > 60) &
                             (cities['Temperature'] < 85) &
                             (cities['Humidity'] < 35) &
                             (cities['Wind'] < 10)].copy()
top_cities.reset_index(drop=True, inplace=True)

top_cities

,City,Longitude,Latitude,Temperature,Humidity,Wind,Cloudiness,Country,Date
0,Albany,-73.9662,42.6001,71.60,24,6.91,1,US,1621119351
1,Jamestown,-79.2353,42.0970,64.40,25,9.22,20,US,1621119969
2,Dalbandin,64.4101,28.8947,77.81,13,1.48,83,PK,1621120001
3,Atar,-13.0499,20.5169,82.42,18,6.20,96,MR,1621120002
4,Ocean City,-74.5746,39.2776,64.40,25,6.91,1,US,1621120013
5,Bethel,-73.4140,41.3712,69.80,28,6.91,20,US,1621120015
6,Palmer,-72.3287,42.1584,75.20,17,9.22,20,US,1621119988
7,Winnipeg,-97.1470,49.8844,75.99,25,1.14,94,CA,1621119782
8,Shelburne,-80.2041,44.0787,68.00,28,2.30,20,CA,1621120044
9,Bud,-86.1758,39.4470,68.00,30,5.75,90,US,1621120055


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# hotel_df = pd.DataFrame()
hotel_df = top_cities
hotel_df['Hotel Name'] = ''
hotel_df

,City,Longitude,Latitude,Temperature,Humidity,Wind,Cloudiness,Country,Date,Hotel Name
0,Albany,-73.9662,42.6001,71.60,24,6.91,1,US,1621119351,
1,Jamestown,-79.2353,42.0970,64.40,25,9.22,20,US,1621119969,
2,Dalbandin,64.4101,28.8947,77.81,13,1.48,83,PK,1621120001,
3,Atar,-13.0499,20.5169,82.42,18,6.20,96,MR,1621120002,
4,Ocean City,-74.5746,39.2776,64.40,25,6.91,1,US,1621120013,
5,Bethel,-73.4140,41.3712,69.80,28,6.91,20,US,1621120015,
6,Palmer,-72.3287,42.1584,75.20,17,9.22,20,US,1621119988,
7,Winnipeg,-97.1470,49.8844,75.99,25,1.14,94,CA,1621119782,
8,Shelburne,-80.2041,44.0787,68.00,28,2.30,20,CA,1621120044,
9,Bud,-86.1758,39.4470,68.00,30,5.75,90,US,1621120055,


In [6]:
params = {'radius': '5000',
          'type': 'lodging',
          'key': g_key}

url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'


for index,row in top_cities.iterrows():
    params['location'] = f'{row.Latitude},{row.Longitude}'
    response = requests.get(url, params=params).json()
    try:
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'City'] = row.City
        hotel_df.loc[index, 'Country'] = row.Country
        hotel_df.loc[index, 'Latitude'] = row.Latitude
        hotel_df.loc[index, 'Longitude'] = row.Longitude
    except:
        print(f'hotel not found in {row.City}.')
        pass
    
# hotel_df

def movecol(hotel_df, cols_to_move=[], ref_col='', place='After'):
    
    cols = hotel_df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(hotel_df[seg1 + seg2 + seg3])

hotel_df = movecol(hotel_df, 
             cols_to_move=['Hotel Name','Country'], 
             ref_col='City',
             place='Before')

hotel_df


hotel not found in Albany.
hotel not found in Kamskiye Polyany.
hotel not found in Avera.


,Hotel Name,Country,City,Longitude,Latitude,Temperature,Humidity,Wind,Cloudiness,Date
0,,US,Albany,-73.9662,42.6001,71.60,24,6.91,1,1621119351
1,Hampton Inn & Suites Jamestown,US,Jamestown,-79.2353,42.0970,64.40,25,9.22,20,1621119969
2,Sanjrani House,PK,Dalbandin,64.4101,28.8947,77.81,13,1.48,83,1621120001
3,Oumou Elghoura,MR,Atar,-13.0499,20.5169,82.42,18,6.20,96,1621120002
4,Biscayne Suites,US,Ocean City,-74.5746,39.2776,64.40,25,6.91,1,1621120013
5,Courtyard by Marriott Danbury,US,Bethel,-73.4140,41.3712,69.80,28,6.91,20,1621120015
6,Wedgewood Motel,US,Palmer,-72.3287,42.1584,75.20,17,9.22,20,1621119988
7,"Humphry Inn & Suites, Downtown Winnipeg",CA,Winnipeg,-97.1470,49.8844,75.99,25,1.14,94,1621119782
8,The Retreat at the Farm,CA,Shelburne,-80.2041,44.0787,68.00,28,2.30,20,1621120044
9,Ted McQuinn Trust,US,Bud,-86.1758,39.4470,68.00,30,5.75,90,1621120055


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))